In [1]:
import ipywidgets
import IPython
import unray
import dolfin
import numpy as np

from dolfin import *
from unray import Data, DataDisplay, render
from IPython.display import display

print(ipywidgets.__version__)
print(IPython.__version__)
print(unray.__version__)


def create_wave_mesh(N, M, thickness, xfreq, yfreq, xamp, yamp):
    mesh = UnitCubeMesh(N, N, M)
    print(mesh.num_cells(), mesh.num_vertices())
    coordinates = mesh.coordinates()
    coordinates[:,2] *= thickness
    coordinates[:,2] += xamp*np.sin(3.14159*xfreq*coordinates[:,0])
    coordinates[:,2] += yamp*np.sin(3.14159*yfreq*coordinates[:,1])
    return mesh

#mesh = UnitCubeMesh(2, 2, 2)
mesh = UnitCubeMesh(1, 1, 1)
#mesh = create_wave_mesh(32, 4, 0.125, 4.0, 3.0, 0.1, 0.05)
#mesh = Mesh("data/heart_mesh.xml.gz")
#mesh = Mesh("data/dog_mesh_97k.xml.gz")
#mesh = UnitCubeMesh(1, 1, 1)
#mesh = UnitTetrahedronMesh()


r = 0
for i in range(r):
    mesh = refine(mesh)
print(mesh.num_cells(), mesh.num_vertices())
V = FunctionSpace(mesh, "P", 1)
f0 = interpolate(Expression("1.0", degree=1), V)
f = interpolate(Expression("x[0]", degree=1), V)
g = interpolate(Expression("x[1]", degree=1), V)
mf = CellFunctionSizet(mesh)

mf.set_all(0)
class SD(SubDomain):
    def inside(self, x, on_boundary):
        return ((x[0]-5)**2 + (x[1]-0.3)**2 + (x[2]-0.3)**2) < 3**2
SD().mark(mf, 10)


cells = mesh.cells()
coordinates = mesh.coordinates()
cell_indicators = np.asarray(mf.array(), dtype="int32")
density = f.compute_vertex_values()
emission = g.compute_vertex_values()
density_lut = None
emission_lut = None

#density_lut = np.asarray([1.0, 1.0, 0.0, 0.0, 1.0, 1.0], dtype="float32")
#density_lut = np.asarray([0.0, 0.0, 1., 1., 0.0, 0.0], dtype="float32")
#density_lut = np.asarray([1, 1, 1, 1], dtype="float32")
density_lut = np.asarray([0.2, .4, .6, .8], dtype="float32")

emission_lut = np.asarray([[1,1,0], [1,.3,.6], [1,.6,.3], [1,1,0]], dtype="float32")
#emission_lut = np.asarray([[1,1,0], [0,1,0], [0,0,1], [1,1,1]], dtype="float32")
#emission_lut=np.asarray([[1,1,.9], [0,.5,0], [.5,.5,.5], [0,1,1]], dtype="float32")


mesh.num_cells(), [(coordinates[:,i].min(), coordinates[:,i].max()) for i in (0,1,2)]

7.0.0a4
6.1.0
0.1.0.dev
6 8


(6, [(0.0, 1.0), (0.0, 1.0), (0.0, 1.0)])

In [2]:
figs = []
#for method in ["xray", "xray2"]:
cells2 = cells
cells2 = cells[:1,:]
for method in ["xray2"]:
    fig = render(coordinates, cells2,
                 density=density, emission=emission,
                 density_lut=density_lut,
                 emission_lut=emission_lut,
                 width=960, height=960,
                 #width=400, height=400,
                 method=method)
    fig.animate = True
    figs.append(fig)
box = ipywidgets.HBox(figs)
box

A Jupyter Widget

In [3]:
for fig in figs:
    fig.animate = False

In [ ]:
for fig in figs:
    fig.animate = True